# Cruce de datos entre base de datos GDC y methbank

En el siguiente notebook se cruzan las bases de datos mencionadas, obteniendo un dataset unificado simple (menos columnas y sin valores faltantes).

In [1]:
import sys
from pathlib import Path
import os

src_path = str(Path.cwd().parents[0] / "src")
if src_path not in sys.path:
    sys.path.append(src_path)

data_raw_path = str(Path.cwd().parents[0] / "data" / "raw")
data_path = str(Path.cwd().parents[0] / "data" )

from get_data import create_folders
import pandas as pd

In [15]:
def read_methbank_data(filename, rows_to_read):
  '''
  - filename: Nombre del archivo .gz en donde está la data del methbank
  - cols_to_read: 
  '''
  mapper = {x:False for x in rows_to_read}

  breast_f = pd.read_csv(filename, sep='\t', compression='gzip',
                           skiprows = lambda x: mapper.get(x-1, True),
                             header=None, quotechar='"', on_bad_lines='skip')
  breast_f_columns = list(breast_f.iloc[:, 0])
  breast_f = breast_f.iloc[:, 1:].T
  breast_f.columns = breast_f_columns

  return breast_f

Para evitar un uso ineficiente de memoria, se leen primero los datos correspondientes a los identificadores de sitios CpG

In [7]:
# Revisar columnas en común
gdc_cols = pd.read_csv(f"{data_raw_path}/brca_27k.csv", sep='\t', nrows=1, header = 0) #pd.read_csv('data_gdc.csv', nrows = 1, header=0)
methbank_cols = pd.read_csv(f"{data_raw_path}/breast_F.txt.gz", compression='gzip', sep="\t", quotechar='"', on_bad_lines='skip', usecols= ['probe'])
common_cols = list(set(methbank_cols['probe']) & (set(gdc_cols.columns)))

In [14]:
print(f"Número de CpG en común: {len(common_cols)}")

Número de CpG en común: 25978


In [16]:
# Sitios CpG están en columna, se determinan que filas contienen los CpG en común
rows_to_read = methbank_cols[methbank_cols['probe'].isin(list(common_cols))].index
df_methbank = read_methbank_data(f"{data_raw_path}/breast_F.txt.gz", rows_to_read)

In [18]:
df_gdc = pd.read_csv(f"{data_raw_path}/brca_27k.csv", sep="\t")

In [20]:
# Toda la data del methbank es tejido normal, se añade variable equivalente a la del gdc
df_methbank['sample_type'] = 'Solid Tissue Normal'
# Se unen dataframes
df_union = pd.concat([df_methbank, df_gdc[common_cols + ['sample_type']]], ignore_index=True)
#df_union = df_union.dropna(axis=1)

In [25]:
# No se usan muestras metastaticas
df_union = df_union[df_union['sample_type']!='Metastatic']

In [27]:
# Quitar sitios CpG con nulos
df_union = df_union.dropna(axis=1)

In [29]:
print(df_union.shape)
data_preprocessed_path = f"{data_path}/preprocessed"
create_folders(data_preprocessed_path)
# Guardar datos
df_union.to_csv(f"{data_preprocessed_path}/gdc_methbank.csv", index=False)

(804, 15775)
